In [2]:
import os
import numpy as np
import pandas as pd
from glob import glob

from dipy.reconst.dti import fractional_anisotropy

In [3]:
ds_dir = os.path.join('/Users/KRS228/data/maastricht_dwi/',
                      'preprocessed/dsi_studio/')
kmeans_dir = os.path.join(ds_dir, 'kmeans')

# define and create (if necessary) output directory
out_dir = os.path.join(kmeans_dir, 'streamline_count')
os.makedirs(out_dir, exist_ok=True)

In [4]:
# define subject list
preprocessed_dir=os.path.abspath('/Users/KRS228/data/maastricht_dwi/preprocessed/01_diff_preprocessed')
subid_list = [os.path.basename(x) for x in sorted((glob(preprocessed_dir+'/*')))]

In [5]:
# create (non-tensor) FA dataframe
roifa_df = pd.DataFrame(index=subid_list)

In [6]:
# define IC ROI list and subdivision codes
roi_list = ['L_IC', 'R_IC']
subdiv_list = ['c', 'd', 'x']

In [7]:
for subid in subid_list:
    for roi in roi_list:
        for kx in range(2,6):
            
            # generate output filename
            out_fname = '%s_%s_kmeans-%d_count-streamlines.csv'%(subid, roi, kx)
            out_fpath = os.path.join(out_dir, out_fname)
            
            # create streamline count matrix (clusters-by-subdivisions)
            sub_sl_mat = np.zeros((kx, len(subdiv_list)))

            for i_subdiv in range (0, len(subdiv_list)):
                
                # create subdiv label by combining IC ROI with subdiv code
                subdiv = roi + subdiv_list[i_subdiv]
                
                # here's where the data are for this subject/IC/k
                subdiv_k_dir = os.path.join(kmeans_dir, 
                                            '%s_%s_kmeans-%d'%(subid, roi, kx),
                                            subdiv)
                # for each cluster, grab the number of streamlines 
                # that hit the current IC subdivision
                for cluster in range(0, kx):
                    cluster_fpath = os.path.join(subdiv_k_dir, 
                                                 '%s_%s_kmeans-%d_cluster%d.txt'%(subid, subdiv, kx, cluster))
                    
                    # count the number of lines in the file (aka number of streamlines)
                    try:
                        num_lines = sum(1 for line in open(cluster_fpath))
                        sub_sl_mat[cluster, i_subdiv] = num_lines
                    except OSError: 
                        # if file doesn't exist, then there were no streamlines to that cluster
                        continue

            # save intermediate files
            #np.savetxt(out_fpath, sub_sl_mat)
            
            # compute (non-tensor) fractional anisotropy
            sub_fa = fractional_anisotropy(sub_sl_mat)
            mean_fa = np.mean(sub_fa)
            roi_k_label = '%s_kmeans-%d'%(roi, kx)
            roifa_df.loc[subid, roi_k_label] = mean_fa

In [8]:
roifa_df

,L_IC_kmeans-2,L_IC_kmeans-3,L_IC_kmeans-4,L_IC_kmeans-5,R_IC_kmeans-2,R_IC_kmeans-3,R_IC_kmeans-4,R_IC_kmeans-5
S01,0.701496,0.771928,0.761988,0.862666,0.682793,0.742240,0.745431,0.736159
S02,0.675932,0.735615,0.761475,0.799243,0.549129,0.634880,0.719185,0.000000
S03,0.443477,0.295651,0.337108,0.089969,0.617935,0.786871,0.816076,0.838286
S05,0.669853,0.753108,0.797581,0.814488,0.511721,0.579456,0.747080,0.767116
S06,0.467655,0.519608,0.528225,0.700396,0.753976,0.670015,0.772938,0.740202
S07,0.623018,0.759156,0.824894,0.836378,0.626321,0.647829,0.488475,0.731648
S08,0.695298,0.756172,0.605701,0.822335,0.771321,0.800129,0.746829,0.771744
S09,0.615267,0.596737,0.698219,0.745434,0.513332,0.460194,0.764561,0.566977
S10,0.606600,0.721715,0.541286,0.614414,0.563853,0.786142,0.748562,0.823039
S11,0.542558,0.556211,0.467505,0.665033,0.721265,0.677424,0.754154,0.850379


In [10]:
# get the across-subject mean FA per IC per n_k
roifa_df.mean(axis=0)

L_IC_kmeans-2    0.604115
L_IC_kmeans-3    0.646590
L_IC_kmeans-4    0.632398
L_IC_kmeans-5    0.695036
R_IC_kmeans-2    0.631165
R_IC_kmeans-3    0.678518
R_IC_kmeans-4    0.730329
R_IC_kmeans-5    0.682555
dtype: float64

In [17]:
from scipy.stats import f_oneway, kruskal

In [14]:
f_oneway(roifa_df.iloc[0], roifa_df.iloc[1], roifa_df.iloc[2], roifa_df.iloc[3])

F_onewayResult(statistic=1.9850121654224295, pvalue=0.1390714122818535)

In [15]:
f_oneway(roifa_df.iloc[4], roifa_df.iloc[5], roifa_df.iloc[6], roifa_df.iloc[7])

F_onewayResult(statistic=2.2167235627309965, pvalue=0.10826088562807842)

In [44]:
f_oneway(roifa_df.iloc[0], roifa_df.iloc[1], roifa_df.iloc[2], roifa_df.iloc[3], roifa_df.iloc[4], roifa_df.iloc[5], roifa_df.iloc[6], roifa_df.iloc[7])

F_onewayResult(statistic=1.812128877755439, pvalue=0.10291236730375136)

In [43]:
f_oneway(roifa_df.iloc[0]+roifa_df.iloc[4], roifa_df.iloc[1]+roifa_df.iloc[5], roifa_df.iloc[2]+roifa_df.iloc[6], roifa_df.iloc[3]+roifa_df.iloc[7])

F_onewayResult(statistic=0.36862587945234343, pvalue=0.7762003470110113)

In [38]:
kruskal(roifa_df.iloc[0]+roifa_df.iloc[4], roifa_df.iloc[1]+roifa_df.iloc[5], roifa_df.iloc[2]+roifa_df.iloc[6], roifa_df.iloc[3]+roifa_df.iloc[7])

KruskalResult(statistic=0.5028409090909065, pvalue=0.918266834488518)